# La gestion des modules en Python

## Module ou *package* ?

**Module :** fichier qui regroupe plusieurs fonctions.

**Package :** répertoire qui rassemble plusieurs modules.

Dans [le précédent *notebook*](1.allocine-get-movies-id.ipynb#Optimiser), nous avons déplacé une fonction dans un fichier séparé intitulé *harvest.py*. Par cette simple opération, nous avons créé un module qu’il est possible d’importer dans un script avec la commande `import` plus le nom du fichier sans l’extension :

In [ ]:
import harvest

**Comment ça ?!** `ModuleNotFoundError` ?

Python a raison, il s’attendait à trouver un fichier *harvest.py* au même niveau que ce *notebook*, c’est-à-dire dans le même répertoire, alors qu’il est rangé dans un sous-dossier *scrape*.

Ce sous-dossier *scrape* est ce que l’on appelle un *package* et un *package* peut renfermer plusieurs modules.

## Importations et espace de noms

In [ ]:
import scrape

On pourrait supposer que cette commande donne accès à tous les modules qui dépendent de `scrape` mais, en l’état, aucune possibilité d’accéder à la fonction `get_html_from_url()` définie dans *harvest* :

In [ ]:
# All these commands fail

# html = get_html_from_url('http://www.llf.cnrs.fr/')
# html = harvest.get_html_from_url('http://www.llf.cnrs.fr/')
# html = scrape.harvest.get_html_from_url('http://www.llf.cnrs.fr/')

Une solution consiste à importer directement le module `harvest` du *package* `scrape` :

In [ ]:
import scrape.harvest

Puis à appeler la fonction en passant par tous les espaces de nommage :

In [ ]:
html = scrape.harvest.get_html_from_url('http://www.llf.cnrs.fr/')

### Alias

Plutôt fastidieux d’appeler une fonction d’aussi loin, non ? Heureusement, il existe les alias :

In [ ]:
import scrape.harvest as sc

html = sc.get_html_from_url('http://www.llf.cnrs.fr/')

### Importation complète

On peut aussi accéder directement aux fonctions sans l’espace de noms :

In [ ]:
from scrape.harvest import *

html = get_html_from_url('http://www.llf.cnrs.fr/')

### Importation spécifique

L’importation totale des fonctions d’un module présente deux risques :

- charger des fonctions inutiles pour le script en cours ;
- l’étoile `*` masque le nom des fonctions.

Une autre méthode consiste à puiser une fonction spécifique dans un module :

In [ ]:
from scrape.harvest import get_html_from_url

html = get_html_from_url('http://www.llf.cnrs.fr/')

## Organiser son *package*

Dans la pratique, on préférera toujours conserver dans la syntaxe d’appel aux fonctions utilisateurs le lien de dépendance à un espace de noms (module) :

In [ ]:
from scrape import harvest

html = harvest.get_html_from_url('http://www.llf.cnrs.fr/')

De cette manière, si l’on définissait dans un module une fonction qui porte le même nom qu’une autre fonction d’un autre module, on éviterait tout risque de collision.

### Des fichiers thématiques

Un *package* s’organise autour de fichiers regroupant des fonctions qui servent elles-mêmes un objectif commun. On peut imaginer des modules avec des noms comme : `core`, `utils`, `tests`, `data`…

### Un sommaire pour le *package*

Il existe un fichier un peu particulier que l’on a coutume de placer en-tête de tout *package*, et ce même s’il est devenu optionnel avec les mises à jour successives de python : *\_\_init\_\_.py*

Ce fichier est utilisé à l’initialisation du paquet et peut se concevoir comme un sommaire du contenu. Il est chargé de lister l’ensemble des modules qu’il contient.

Créez un fichier *\_\_init\_\_.py* à la racine du répertoire *scrape* avec juste ces quelques lignes :

```py
__name__ = 'Scrape: the Web-corpus maker'
__version__ = 'v1.0b'

from .utils import get_html_from_url
```

Ce qui permet ensuite de raccourcir les appels aux fonctions de tous les modules dépendants, comme s’il s’agissait de méthodes du *package* :

In [ ]:
import scrape

html = scrape.get_html_from_url('http://www.llf.cnrs.fr/')

## Les variables spéciales : l’exemple de `__name__`

Vous avez sans doute remarqué l’utilisation de variables encadrées par des doubles underscores :
- `__name__`
- `__version__`

Ce sont des variables spéciales qui permettent d’indiquer certaines informations à Python. Elles sont facultatives et ne servent que dans des cas particuliers.

Prenons l’exemple de la variable `__name__` que l’on trouve dans l’expression :

```py
if __name__ == '__main__':
    # Main program
```

Lorsque Python lit un fichier `.py`, il réalise deux opérations :
- exécuter une instruction similaire à : `__name__ = '__main__'`
- exécuter le code du script

D’un point du vue pratique, observez le fichier [*utils.py*](scrape/utils.py), qui dispose d’un morceau de code :
```py
if __name__ == '__main__':
    """Usage example.
    Works only when used as standalone.
    """
    url = "http://www.llf.cnrs.fr/ita/"
    html = get_html_from_url(url)
    anchors = parse_html_by_class(html, '.liste-membres .field-content a')
    for anchor in anchors:
        print(anchor.get_text())
```

Ce morceau de code n’est exécuté que lorsque le script *utils.py* est appelé tout seul (*standalone*). Téléchargez-le et lancez-le dans une console :

```shell
$ python utils.py
```

Vous verrez une liste de noms s’afficher.

En revanche, lorsque vous importez le module depuis un script, la liste de noms ne s’affiche pas !

In [ ]:
import scrape.harvest

L’explication est simple : lorsque le script est importé en tant que module, son `__name__` prend la valeur du fichier. En l’occurrence, le script *harvest.py* a pour nom `harvest` et pas `__main__`.

Le nom `__main__` est réservé au script principal : ce *notebook* !

Faites un test :

1. Éditez le fichier *harvest.py*
2. Retirez l’instruction `if __name__ = '__main__':`
3. Relancez le *kernel*
4. Exécutez de nouveau l’instruction `import scrape.harvest`

Cette fois-ci, alors que l’importation du module s’effectue, le code qui sert d’exemple à l’utilisation du module s’exécute et la liste de noms apparaît…

La condition `if __name__ = '__main__':` est par conséquent impérative pour éviter tout effet de bord dans un script !